In [0]:

/*
Segment products into cost ranges and 
count how many products fall into each segment.
*/
with product_segments (
  SELECT 
    product_key,
    prodcut_name,
    cost,
    case 
      when cost < 100 then 'Below 100'
      when cost between 100 and 500 then '100-500'
      when cost between 500 and 1000 then '500-100'
      else 'Above 1000'
    end as cost_range
  FROM datawarehouse.gold.dim_prodcut  GROUP BY product_key, prodcut_name, cost

)
select 
  cost_range,
  count(product_key) as total_products
from product_segments
group by cost_range
order by total_products desc;



/*
Group customers into three segments based on their spending begavior:
  - VIP: Customer with at least 12 months of history and spending more then $5,000
  - Regular: Customers with at least 12 months of history but spending $5000 or less.
  - New: Customers with a lifespan less than 12 months.
And find the total number of customers by each group
*/
with customer_segments as (
  select 
    c.customer_key,
    concat(c.first_name, ' ', c.last_name) as customer_name,
    min(s.order_date) as first_order_date,
    max(s.order_date) as last_order_date,
    sum(s.sales_amount) as total_sales,
    case 
      when date_diff(MONTH, first_order_date, last_order_date) >= 12 and total_sales > 5000 then 'VIP'
      when date_diff(MONTH, first_order_date, last_order_date) >= 12 and total_sales <= 5000 then 'Regular'
      else 'New' 
    end as customer_segment
  from datawarehouse.gold.fact_sales s
  left join datawarehouse.gold.dim_customer c
  on s.customer_key = c.customer_key
  group by c.customer_key, customer_name
  order by min(s.order_date)
)
select 
  customer_segment,
  count(*) as total_customers
from customer_segments
group by customer_segment
order by total_customers desc;






